# Stencil Numba GPU SD18 MPI

```
source $SCR/env2/etc/profile.d/conda.sh
conda activate $SCR/env2
conda activate --stack $SCR/env3
```

In [1]:
%%bash
gfortran --version | head -1
mpiexec --version | head -2
conda list '^(anaconda|python)$'

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39)
HYDRA build details:
    Version:                                 3.3.2
# packages in environment at /scratch/ampemi/xxxx.xxxx/env3:
#
# Name                    Version                   Build  Channel
python                    3.9.4                hdb3f193_0  


In [1]:
from numba import cuda
print(cuda.gpus)

<Managed Device 0>, <Managed Device 1>, <Managed Device 2>, <Managed Device 3>


In [12]:
%%writefile ng.py
import numpy as np, math
from time import time
from mpi4py import MPI
from numba import cuda, njit, prange, config

# parameters
n            = 4800    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations
# initialize three heat sources
nsources     = 3       # number of sources of energy
sources      = np.zeros((nsources, 2), np.int16)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
# initialize the data arrays
anew         = np.zeros((n + 2, n + 2), np.float64)
aold         = np.zeros((n + 2, n + 2), np.float64)

# configure blocks & grids
## set the number of threads in a block
threads_per_block = (16, 16)    # based on trial and error
## calculate the number of thread blocks in the grid
blocks_per_grid_x = math.ceil(aold.shape[0] / threads_per_block[0])
blocks_per_grid_y = math.ceil(aold.shape[1] / threads_per_block[1])
blocks_per_grid   = (blocks_per_grid_x, blocks_per_grid_y)

# computationally intensive core
@cuda.jit
def kernel(A, B):
    n = A.shape[0] - 1
    i, j = cuda.grid(2)
    if (i > 0 and j > 0) and (i < n and j < n) :
        A[i,j]=B[i,j]*.5+(B[i-1,j]+B[i+1,j]+B[i,j-1]+B[i,j+1])*.125

# start of main routine

###mpi4py###
comm  = MPI.COMM_WORLD            # MPI default communicator
size  = comm.Get_size()           # MPI size
rank  = comm.Get_rank()           # MPI rank
name  = MPI.Get_processor_name()  # core hostname (eg sdumont3170)

#Only 2 processes per node are selected via Slurm. Within a node, color 
#rank 0 corresponds to the first process of this node, and color rank 1 
#corresponds to the second process of this node, and the other nodes are 
#similar. Example:
#  node      rank  color rank
#----------- ----  ----------
#sdumont3170   0        0
#sdumont3170   1        1
#sdumont3171   2        0
#sdumont3171   3        1
#sdumont3172   4        0
#sdumont3172   5        1
#sdumont3173   6        0
#sdumont3173   7        1
for i, c in enumerate(name) :     # find first digit in hostname
    if c.isdigit() :
        break
mcol  = int(name[i:])             # extract number from hostname
scomm = comm.Split(color = mcol)  # new communicator for the node
crank = scomm.Get_rank()          # get the node color rank

###numba.cuda###
#In this implementation, Slurm is configured to run only 2 processes on 
#each node. For each of these processes (cores), a single GPU is 
#associated. Thus, within a node, color rank 0 is associated with GPU 0, 
#and color rank 1 is associated with GPU 1.
#cuda.select_device(crank)         # 'color rank' 0 = 'gpu id' 0, etc.
cuda.select_device(rank%4)         # for mpiexec run
cid = cuda.current_context().device.id

# time measurement for rank 0
if not rank :
    tt = -time()    # rank 0 time
    tk = 0          # accumulate kernel time
    tc = 0          # accumulate communication time
    te = 0          # energy insertion time

# determine my coordinates (x,y)
pdims = MPI.Compute_dims(size, 2)
px    = pdims[0]
py    = pdims[1]
rx    = rank % px
ry    = rank // px

# determine my four neighbors
north = (ry - 1) * px + rx
if (ry - 1) < 0 :
    north = MPI.PROC_NULL
south = (ry + 1) * px + rx
if (ry + 1) >= py :
    south = MPI.PROC_NULL
west = ry * px + rx - 1
if (rx - 1) < 0 :
    west = MPI.PROC_NULL
east = ry * px + rx + 1
if (rx + 1) >= px :
    east = MPI.PROC_NULL

# decompose the domain
bx   = n // px          # block size in x
by   = n // py          # block size in y
offx = rx * bx + 1      # offset in x
offy = ry * by + 1      # offset in y

# sources in my area, local to my rank
locnsources = 0
locsources  = np.empty((nsources, 2), np.int16)

# determine which sources are in my patch
for i in range(nsources) :
    locx = sources[i, 0] - offx
    locy = sources[i, 1] - offy
    if(locx >= 0 and locx <= bx and locy >= 0 and locy <= by) :
        locsources[locnsources, 0] = locx
        locsources[locnsources, 1] = locy
        locnsources += 1

# working arrays with 1-wide halo zones
anew = np.zeros((bx+2, by+2), np.float64)
aold = np.zeros((bx+2, by+2), np.float64)

# system total heat
rheat = np.zeros(1, np.float64)
bheat = np.zeros(1, np.float64)

# copy the first arrays to the device
if not rank : tc -= time()
anew_global_mem    = cuda.to_device(anew)
aold_global_mem    = cuda.to_device(aold)
#sources_global_mem = cuda.to_device(sources)
if not rank : tc += time()
   
# main loop
for _ in range(0, niters, 2) :

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(aold[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(aold[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(aold[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(aold[1:bx+1, 1], dest=west, tag=1)
    # wait for the end of communication
    if north != MPI.PROC_NULL :
        s1.wait()
        aold[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        aold[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        aold[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        aold[1:bx+1, 0] = r4.wait()

    # copy the received array to the device
    if not rank : tc -= time()
    aold_global_mem = cuda.to_device(aold)
    if not rank : tc += time()

    # update grid
    if not rank : tk -= time()
    kernel[blocks_per_grid, threads_per_block](
        anew_global_mem, aold_global_mem)
    if not rank : tk += time()

    # copy the result back to the host
    if not rank : tc -= time()
    anew = anew_global_mem.copy_to_host()
    if not rank : tc += time()

    # refresh heat sources
    if not rank : te -= time()
    for i in range(locnsources) :
        anew[locsources[i, 0]-1, locsources[i, 1]-1] += energy
    if not rank : te += time()

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(anew[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(anew[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(anew[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(anew[1:bx+1, 1], dest=west, tag=1)
    # wait for the end of communication
    if north != MPI.PROC_NULL :
        s1.wait()
        anew[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        anew[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        anew[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        anew[1:bx+1, 0] = r4.wait()

    # copy the received array to the device
    if not rank : tc -= time()
    anew_global_mem = cuda.to_device(anew)
    if not rank : tc += time()

    # update grid
    if not rank : tk -= time()
    kernel[blocks_per_grid, threads_per_block](
        aold_global_mem, anew_global_mem)
    if not rank : tk += time()

    # copy the result back to the host
    if not rank : tc -= time()
    aold = aold_global_mem.copy_to_host()
    if not rank : tc += time()
        
    # refresh heat sources
    if not rank : te -= time()
    for i in range(locnsources) :
        aold[locsources[i, 0]-1, locsources[i, 1]-1] += energy
    if not rank : te += time()

# end for

# get final heat in the system
bheat[0] = np.sum(aold[1:-1, 1:-1])
comm.Reduce(bheat, rheat)

# show the result
print(f"3. {name:11s}   {rank:02d}    {crank:02d}   {cid:02d}")
if not rank :
    tt += time()
    print( "1. hostname   rank crank  cid")
    print( "2. ----------- ---- ----- ----")
    print( "4. ---------------------------")
    print(f"5. Heat:{rheat[0]:.4f}", end=", ")
    print(f"TT:{tt:.4f}", end=", ")
    print(f"TK:{tk:.4f}", end=", ")
    print(f"TC:{tc:.4f}", end=", ")
    print(f"TE:{te:.4f}", end=", ")
    print(f"MPI:{size}")

Overwriting ng.py


### Antigo, sem TE:

In [6]:
! mpiexec -n 1 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
4. ---------------------------
5. Heat:1500.0000, TT:53.6296, KT:0.2433, CT:53.3408, MPISize:1


In [8]:
! mpiexec -n 1 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
4. ---------------------------
5. Heat:1500.0000, TT:50.0480, KT:0.2392, CT:49.7663, MPISize:1


In [9]:
! mpiexec -n 1 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
4. ---------------------------
5. Heat:1500.0000, TT:50.7709, KT:0.2375, CT:50.4876, MPISize:1


In [7]:
! mpiexec -n 4 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
4. ---------------------------
5. Heat:1500.0000, TT:15.6257, KT:0.2501, CT:14.8573, MPISize:4


In [10]:
! mpiexec -n 4 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
4. ---------------------------
5. Heat:1500.0000, TT:14.6767, KT:0.2524, CT:12.5157, MPISize:4


In [11]:
! mpiexec -n 4 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
4. ---------------------------
5. Heat:1500.0000, TT:14.9962, KT:0.2638, CT:14.5452, MPISize:4


In [5]:
! mpiexec -n 9 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
4. ---------------------------
5. Heat:1500.0000, TT:6.7512, KT:0.3051, CT:5.8022, MPISize:9


In [12]:
! mpiexec -n 9 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
4. ---------------------------
5. Heat:1500.0000, TT:6.9152, KT:0.3125, CT:6.1009, MPISize:9


In [13]:
! mpiexec -n 9 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
4. ---------------------------
5. Heat:1500.0000, TT:7.1948, KT:0.2750, CT:5.5385, MPISize:9


In [14]:
! mpiexec -n 16 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
4. ---------------------------
5. Heat:1500.0000, TT:7.1660, KT:0.2903, CT:3.8720, MPISize:16


In [15]:
! mpiexec -n 16 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
4. ---------------------------
5. Heat:1500.0000, TT:6.2568, KT:0.2906, CT:4.4598, MPISize:16


In [16]:
! mpiexec -n 16 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
4. ---------------------------
5. Heat:1500.0000, TT:6.0522, KT:0.3049, CT:4.4625, MPISize:16


In [17]:
! mpiexec -n 36 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
3. sdumont18     16    16   00
3. sdumont18     17    17   01
3. sdumont18     18    18   02
3. sdumont18     19    19   03
3. sdumont18     20    20   00
3. sdumont18     21    21   01
3. sdumont18     22    22   02
3. sdumont18     23    23   03
3. sdumont18     24    24   00
3. sdumont18     25    25   01
3. sdumont18     26    26   02
3. sdumont18     27    27   03
3. sdumont18     28    28   00
3. sdumont18     29    29   01
3. sdumon

In [18]:
! mpiexec -n 36 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
3. sdumont18     16    16   00
3. sdumont18     17    17   01
3. sdumont18     18    18   02
3. sdumont18     19    19   03
3. sdumont18     20    20   00
3. sdumont18     21    21   01
3. sdumont18     22    22   02
3. sdumont18     23    23   03
3. sdumont18     24    24   00
3. sdumont18     25    25   01
3. sdumont18     26    26   02
3. sdumont18     27    27   03
3. sdumont18     28    28   00
3. sdumont18     29    29   01
3. sdumon

In [19]:
! mpiexec -n 36 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
3. sdumont18     16    16   00
3. sdumont18     17    17   01
3. sdumont18     18    18   02
3. sdumont18     19    19   03
3. sdumont18     20    20   00
3. sdumont18     21    21   01
3. sdumont18     22    22   02
3. sdumont18     23    23   03
3. sdumont18     24    24   00
3. sdumont18     25    25   01
3. sdumont18     26    26   02
3. sdumont18     27    27   03
3. sdumont18     28    28   00
3. sdumont18     29    29   01
3. sdumon

## Novo, com TE:

In [2]:
! mpiexec -n 1 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
4. ---------------------------
5. Heat:1500.0000, TT:48.3958, TK:1.9448, TC:46.4033, TE:0.0178, MPI:1


In [3]:
! mpiexec -n 1 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
4. ---------------------------
5. Heat:1500.0000, TT:46.7918, TK:0.2551, TC:46.4928, TE:0.0160, MPI:1


In [4]:
! mpiexec -n 1 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
4. ---------------------------
5. Heat:1500.0000, TT:53.4258, TK:0.3811, TC:52.9990, TE:0.0169, MPI:1


## 4

In [13]:
! mpiexec -n 4 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
4. ---------------------------
5. Heat:1500.0000, TT:15.1065, TK:0.2787, TC:14.5037, TE:0.0158, MPI:4


In [14]:
! mpiexec -n 4 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
4. ---------------------------
5. Heat:1500.0000, TT:15.0891, TK:0.2742, TC:14.5493, TE:0.0167, MPI:4


In [15]:
! mpiexec -n 4 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
4. ---------------------------
5. Heat:1500.0000, TT:15.2804, TK:0.2698, TC:14.6051, TE:0.0157, MPI:4


## 9

In [16]:
! mpiexec -n 9 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
4. ---------------------------
5. Heat:1500.0000, TT:6.8329, TK:0.3227, TC:5.3998, TE:0.0125, MPI:9


In [17]:
! mpiexec -n 9 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
4. ---------------------------
5. Heat:1500.0000, TT:6.9153, TK:0.2889, TC:5.8396, TE:0.0116, MPI:9


In [18]:
! mpiexec -n 9 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
4. ---------------------------
5. Heat:1500.0000, TT:6.9853, TK:0.3086, TC:6.1032, TE:0.0132, MPI:9


## 16

In [19]:
! mpiexec -n 16 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
4. ---------------------------
5. Heat:1500.0000, TT:6.6906, TK:0.3937, TC:3.6735, TE:0.0018, MPI:16


In [20]:
! mpiexec -n 16 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
4. ---------------------------
5. Heat:1500.0000, TT:6.8173, TK:0.4118, TC:4.0617, TE:0.0010, MPI:16


In [21]:
! mpiexec -n 16 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
4. ---------------------------
5. Heat:1500.0000, TT:6.6453, TK:0.4068, TC:4.8067, TE:0.0016, MPI:16


## 36

In [22]:
! mpiexec -n 36 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
3. sdumont18     16    16   00
3. sdumont18     17    17   01
3. sdumont18     18    18   02
3. sdumont18     19    19   03
3. sdumont18     20    20   00
3. sdumont18     21    21   01
3. sdumont18     22    22   02
3. sdumont18     23    23   03
3. sdumont18     24    24   00
3. sdumont18     25    25   01
3. sdumont18     26    26   02
3. sdumont18     27    27   03
3. sdumont18     28    28   00
3. sdumont18     29    29   01
3. sdumon

In [23]:
! mpiexec -n 36 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
3. sdumont18     16    16   00
3. sdumont18     17    17   01
3. sdumont18     18    18   02
3. sdumont18     19    19   03
3. sdumont18     20    20   00
3. sdumont18     21    21   01
3. sdumont18     22    22   02
3. sdumont18     23    23   03
3. sdumont18     24    24   00
3. sdumont18     25    25   01
3. sdumont18     26    26   02
3. sdumont18     27    27   03
3. sdumont18     28    28   00
3. sdumont18     29    29   01
3. sdumon

In [24]:
! mpiexec -n 36 python ng.py | sort

1. hostname   rank crank  cid
2. ----------- ---- ----- ----
3. sdumont18     00    00   00
3. sdumont18     01    01   01
3. sdumont18     02    02   02
3. sdumont18     03    03   03
3. sdumont18     04    04   00
3. sdumont18     05    05   01
3. sdumont18     06    06   02
3. sdumont18     07    07   03
3. sdumont18     08    08   00
3. sdumont18     09    09   01
3. sdumont18     10    10   02
3. sdumont18     11    11   03
3. sdumont18     12    12   00
3. sdumont18     13    13   01
3. sdumont18     14    14   02
3. sdumont18     15    15   03
3. sdumont18     16    16   00
3. sdumont18     17    17   01
3. sdumont18     18    18   02
3. sdumont18     19    19   03
3. sdumont18     20    20   00
3. sdumont18     21    21   01
3. sdumont18     22    22   02
3. sdumont18     23    23   03
3. sdumont18     24    24   00
3. sdumont18     25    25   01
3. sdumont18     26    26   02
3. sdumont18     27    27   03
3. sdumont18     28    28   00
3. sdumont18     29    29   01
3. sdumon